In [3]:
#import needed packages

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import spacy
import en_core_web_sm

import re
import string
import os
import sys
import codecs
import argparse
import json
import gzip
import re
from collections import Counter
import numpy as np
from numpy import mean
from numpy import std
from tqdm import tqdm


# Machine learning imports
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import numpy as np

# NLP imports
from spacy.lang.en import English

In [4]:
#format the MyPersonality data

df = pd.read_csv ('MyPersonalityData.csv')

df = df.groupby('#AUTHID').agg({'cNEU':'first', 
                             'STATUS': ', '.join }).reset_index()

data = df.values.tolist()

In [5]:
#format the Essay data

df = pd.read_csv ('Essays.csv')
df = df.groupby('#AUTHID').agg({'cNEU':'first', 
                             str('TEXT'): ', '.join }).reset_index()

essay_data = df.values.tolist()

In [6]:
# Read from the data, returning parallel lists of documents and their labels

def read_and_clean_lines(data):   
    NeuScores = []
    Statuses = []

   
    for line in data:
        
        NeuScore = line[1]
        Status = line[2]
        
        clean_text = re.sub(r"\s+"," ",Status)
        Statuses.append(clean_text)
        NeuScores.append(NeuScore)
        
    #print("Read {} documents".format(len(Statuses)))
    #print("Read {} labels".format(len(NeuScores)))
    return Statuses,NeuScores

def normalize_tokens(tokenlist):
    # Input: list of tokens as strings,  e.g. ['I', ' ', 'saw', ' ', '@psresnik', ' ', 'on', ' ','Twitter']
    # Output: list of tokens where
    #   - All tokens are lowercased
    #   - All tokens starting with a whitespace character have been filtered out
    #   - All handles (tokens starting with @) have been filtered out
    #   - Any underscores have been replaced with + (since we use _ as a special character in bigrams)

    normalized_tokens = tokenlist # ASSIGNMENT: replace with your code
    
    return normalized_tokens

def ngrams(tokens, n):
    # Returns all ngrams of size n in sentence, where an ngram is itself a list of tokens
    return [tokens[i:i+n] for i in range(len(tokens)-n+1)]

def filter_punctuation_bigrams(ngrams):
    # Input: assume ngrams is a list of ['token1','token2'] bigrams
    # Removes ngrams like ['today','.'] where either token is a punctuation character
    # Returns list with the items that were not removed
    punct = string.punctuation
    return [ngram   for ngram in ngrams   if ngram[0] not in punct and ngram[1] not in punct]

def filter_stopword_bigrams(ngrams, stopwords):
    # Input: assume ngrams is a list of ['token1','token2'] bigrams, stopwords is a set of words like 'the'
    # Removes ngrams like ['in','the'] and ['senator','from'] where either word is a stopword
    # Returns list with the items that were not removed
    result = [ngram   for ngram in ngrams   if ngram[0] not in stopwords and ngram[1] not in stopwords]
    return result


# Split on whitespace, e.g. "a    b_c  d" returns tokens ['a','b_c','d']
def whitespace_tokenizer(line):
    return line.split()

In [7]:
# Read a set of stoplist words from data, assuming it contains one word per line
# Return a python Set data structure (https://www.w3schools.com/python/python_sets.asp)

def load_stopwords(filename):
    stopwords = []
    with codecs.open(filename, 'r', encoding='ascii', errors='ignore') as fp:
        stopwords = fp.read().split('\n')
    return set(stopwords)

In [8]:
# Call sklearn's train_test_split function to split the dataset into training items/labels and test items/labels.  

def split_training_set(lines, labels, test_size, random_seed=42):
    X_train, X_test, y_train, y_test = train_test_split(lines, labels, test_size=test_size, random_state=random_seed, stratify=labels)
    #print("Training set label counts: {}".format(Counter(y_train)))
    #print("Test set     label counts: {}".format(Counter(y_test)))
    return X_train, X_test, y_train, y_test

In [27]:
#bagofwords 

def BOW(X, stopwords_arg, analyzefn="word", range=(1,10)):
    training_vectorizer = CountVectorizer(stop_words=stopwords_arg,
                                          analyzer=analyzefn,
                                          lowercase=True,
                                          ngram_range=range)
    
    X_features = training_vectorizer.fit_transform(X)
    return X_features, training_vectorizer

In [19]:
#tf-idf

def TF_IDF(X, stopwords_arg, analyzefn="word", range=(1,4)):
    
    from sklearn.feature_extraction.text import TfidfVectorizer

  
    training_vectorizer = TfidfVectorizer(min_df = 10, max_df = 0.1, stop_words=stopwords_arg,
                                          analyzer=analyzefn,
                                          lowercase=True,
                                          ngram_range=range)
    
    X_features = training_vectorizer.fit_transform(X)
  
    return X_features, training_vectorizer

In [11]:
#Linguistic Inquiry and Word Count

def LIWC(X, range_low, range_high):
    
    import liwc
    parse, category_names = liwc.load_token_parser('liwc2007_updates-DO_NOT_SHARE.dic')
    
    import re
    from collections import Counter
    
    #tokenize the text
    def tokenize(text):
        for match in re.finditer(r'\w+', text, re.UNICODE):
            yield match.group(0)

    data_dict = []
   
    for line in X:
        line = line.lower()
        tokens = tokenize(line)
        
        
        c = Counter(category for token in tokens for category in parse(token))
        
        #use = ['future','feel','sad','cause','home','leisure','verb','tentat','they','posemo','negemo','death','adverb','hear','ingest','swear','percept','we','money','insight','nonfl','see','family','anger','auxverb','inhib','number','past','excl','assent','i','cogmech','anx','friend','bio','ipron','article','you','present','achieve']
        
        #for word in list(c):
            #if word not in use:
                #del c[word]
        
        
        c = Counter({k: c for k, c in c.items() if c >= range_low})
        c = Counter({k: c for k, c in c.items() if c <= range_high})
            
        d = dict(c)
        
        data_dict.append(c)
        
    # Create DictVectorizer object
    dictvectorizer = DictVectorizer(sparse=False)

    # Convert dictionary into feature matrix
    features = dictvectorizer.fit_transform(data_dict)
    
    return features, dictvectorizer
    
    

In [12]:
#empath

def Empath(X):
    
    from empath import Empath
    lexicon = Empath()
    
    import re
    from collections import Counter


    data_dict = []
  
    
    for line in X:
        
        line = line.lower()
        c = lexicon.analyze(line, normalize=True)
        counts = dict(c)
        
        data_dict.append(counts)
        
    
    # Create DictVectorizer object
    dictvectorizer = DictVectorizer(sparse=False)

    # Convert dictionary into feature matrix
    features = dictvectorizer.fit_transform(data_dict)
    
    return features, dictvectorizer

In [13]:
def show_most_informative_features(vectorizer, clf, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [31]:
###################################################################################################################
# Adaptation of the main logistic regression experiment from Assignment 2, to use cross-validation.
###################################################################################################################
def run_experiment(Classifier, Feature_Extraction_Method, input, essay_data, stopwords_file, test_size, num_folds, stratify, random_seed, range_low,range_high):

    # Load stopwords
    stop_words = load_stopwords(stopwords_file)

    #Clean training/test data
    X, y = read_and_clean_lines(input)
    Len_X = len(X)
  
    #Clean essay data
    A, b = read_and_clean_lines(essay_data)
    Len_A = len(A)

    #combine both MyPersonality and Essay data
    Combined = X+A
        
    if Feature_Extraction_Method == "BOW":
        
        Features, training_vectorizer = BOW(Combined, stop_words, "word", range=(1,10))
        Len_F = Features.getnnz()
        X_features= Features[0:Len_X]
        Essay_features = Features[Len_X:Len_F+1]

    if Feature_Extraction_Method == "TF_IDF":
    
        Features, training_vectorizer = TF_IDF(Combined, stop_words, "word", range=(1,3))
        Len_F = Features.getnnz()
        X_features= Features[0:Len_X]
        Essay_features = Features[Len_X:Len_F+1]
      
    if Feature_Extraction_Method == "LIWC":
        
        Features, training_vectorizer = LIWC(Combined, range_low, range_high)
        Len_F = len(Features)
        X_features= Features[0:Len_X]
        Essay_features = Features[Len_X:Len_F+1]
        
    if Feature_Extraction_Method == "Empath":
        
        Features, training_vectorizer = Empath(Combined)
        Len_F = len(Features)
        X_features= Features[0:Len_X]
        Essay_features = Features[Len_X:Len_F+1]
      
    
    #split MyPeronality data
    X_train, X_test, y_train, y_test = split_training_set(X_features, y, test_size=test_size)
   
    # Create a k-fold cross validation object.
    if stratify == False:
        cv = KFold(n_splits=num_folds, random_state=random_seed, shuffle=True)
        

    if stratify == True:
        cv = StratifiedKFold(n_splits=num_folds, random_state=random_seed, shuffle=True)
    
    # create model
    if Classifier == "Logistic Regression":
        model = LogisticRegression(solver='liblinear')
    
    if Classifier == "SVM":
        model = SVC(kernel='linear')
        
    if Classifier == "DecisionTree":
        model = DecisionTreeClassifier()
        
    if Classifier == "NB":
        model = MultinomialNB()
    
    
    # evaluate model
    train_scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
  
    print('TRAINING SET --- Mean: %.3f STD: %.3f' % (mean(train_scores), std(train_scores)))
  
    # fit model
    model.fit(X_train, y_train)
    
    # make predictions on remaining test set
    yhat = model.predict(X_test)
    # evaluate predictions
    accuracy = accuracy_score(y_test, yhat)
    print('Accuracy of Test Set Predictions: %.3f' % (accuracy * 100))
    
    # make predictions on the essay dataset
    xhat = model.predict(Essay_features)
    # evaluate predictions
    accuracy = accuracy_score(b, xhat)
    print('Accuracy of Essay Data Predictions: %.3f' % (accuracy * 100))
    
    
    clf = LogisticRegression(random_state=0).fit(X_train, y_train)
    print(show_most_informative_features(training_vectorizer, clf, n=20))
    
    return accuracy
    
       

In [15]:
#best LIWC ranges for Logistic Regression
run_experiment("Logistic Regression", "LIWC", data, essay_data, "mallet_en_stoplist.txt", 0.1, 5, True, 3, 1, 15) 

run_experiment("Logistic Regression", "LIWC", data, essay_data, "mallet_en_stoplist.txt", 0.1, 5, True, 3, 101, 1000)

run_experiment("Logistic Regression", "LIWC", data, essay_data, "mallet_en_stoplist.txt", 0.9, 5, True, 3, 1, 5) 

run_experiment("Logistic Regression", "LIWC", data, essay_data, "mallet_en_stoplist.txt", 0.9, 5, True, 3, 791, 1000)

0.6044444444444445

In [340]:
#best LIWC ranges for SVM
run_experiment("SVM", "LIWC", data, essay_data, "mallet_en_stoplist.txt", 0.1, 10, True, 3, 1, 120) 

run_experiment("SVM", "LIWC", data, essay_data, "mallet_en_stoplist.txt", 0.1, 10, True, 3, 251, 1000)

#best LIWC ranges for SVM
run_experiment("SVM", "LIWC", data, essay_data, "mallet_en_stoplist.txt", 0.9, 10, True, 3, 1, 45) 

run_experiment("SVM", "LIWC", data, essay_data, "mallet_en_stoplist.txt", 0.9, 10, True, 3, 216, 1000)

TRAINING SET --- Mean: 0.591 STD: 0.031
Accuracy of Test Set Predictions: 64.000
Accuracy of Essay Data Predictions: 50.162
	-0.2714	future         		0.3484	family         
	-0.2450	they           		0.3089	nonfl          
	-0.2305	hear           		0.2904	see            
	-0.2268	leisure        		0.2477	anger          
	-0.2034	humans         		0.2224	number         
	-0.1468	money          		0.2179	excl           
	-0.1457	sad            		0.1714	anx            
	-0.1439	ingest         		0.1292	inhib          
	-0.1249	home           		0.0993	ipron          
	-0.1201	tentat         		0.0900	posemo         
	-0.1150	quant          		0.0843	friend         
	-0.1115	sexual         		0.0659	incl           
	-0.1007	feel           		0.0600	article        
	-0.0945	affect         		0.0461	assent         
	-0.0896	negate         		0.0458	ppron          
	-0.0780	death          		0.0445	negemo         
	-0.0741	we             		0.0436	present        
	-0.0686	cause          		0.0417	body      

C:\Users\conoc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TRAINING SET --- Mean: 0.609 STD: 0.041
Accuracy of Test Set Predictions: 60.000
Accuracy of Essay Data Predictions: 50.527
	-0.0573	ppron          		0.1116	present        
	-0.0509	pronoun        		0.0281	verb           
	-0.0508	preps          		0.0012	funct          
	-0.0452	social         		0.0000	space          
	-0.0394	time           		0.0000	ipron          
	-0.0332	auxverb        		0.0000	i              
	-0.0036	cogmech        		0.0000	conj           
	-0.0010	relativ        		0.0000	article        
	0.0000	adverb         		0.0000	affect         
	0.0000	affect         		0.0000	adverb         
	0.0000	article        		-0.0010	relativ        
	0.0000	conj           		-0.0036	cogmech        
	0.0000	i              		-0.0332	auxverb        
	0.0000	ipron          		-0.0394	time           
	0.0000	space          		-0.0452	social         
	0.0012	funct          		-0.0508	preps          
	0.0281	verb           		-0.0509	pronoun        
	0.1116	present        		-0.0573	ppron       

C:\Users\conoc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TRAINING SET --- Mean: 0.583 STD: 0.083
Accuracy of Test Set Predictions: 63.111
Accuracy of Essay Data Predictions: 50.891
	0.0000	adverb         		0.0207	cogmech        
	0.0000	affect         		0.0203	pronoun        
	0.0000	article        		0.0007	funct          
	0.0000	auxverb        		0.0000	verb           
	0.0000	bio            		0.0000	time           
	0.0000	conj           		0.0000	space          
	0.0000	i              		0.0000	social         
	0.0000	ipron          		0.0000	relativ        
	0.0000	posemo         		0.0000	present        
	0.0000	ppron          		0.0000	preps          
	0.0000	preps          		0.0000	ppron          
	0.0000	present        		0.0000	posemo         
	0.0000	relativ        		0.0000	ipron          
	0.0000	social         		0.0000	i              
	0.0000	space          		0.0000	conj           
	0.0000	time           		0.0000	bio            
	0.0000	verb           		0.0000	auxverb        
	0.0007	funct          		0.0000	article        
	0.0203	pron

0.5089141004862237

In [30]:
run_experiment("Logistic Regression", "BOW", data, essay_data, "mallet_en_stoplist.txt", 0.5, 10, True, 3, 400,600) 
    

	-0.2496	god            		0.2983	black          
	-0.1885	canada         		0.2518	cavs           
	-0.1867	bless          		0.1322	long           
	-0.1867	canada fam     		0.1306	long time      
	-0.1867	canada fam god 		0.1259	rock house     
	-0.1867	canada fam god bless		0.1259	cavs rock house
	-0.1867	fam god        		0.1259	cavs rock      
	-0.1867	fam god bless  		0.1259	cavs cavs rock house
	-0.1867	god bless      		0.1259	cavs cavs rock 
	-0.1849	fam            		0.1259	cavs cavs      
	-0.1697	give           		0.1208	hates          
	-0.1631	happy          		0.1196	rock           
	-0.1563	things         		0.1155	marea hates long time
	-0.1535	life           		0.1155	marea hates long
	-0.1516	mornings       		0.1155	marea hates    
	-0.1477	magic          		0.1155	marea          
	-0.1459	magic things   		0.1155	hates long time
	-0.1459	magic things give		0.1155	hates long     
	-0.1459	mornings winter		0.1155	facebook marea hates long time
	-0.1459	mornings winter magic		0.1

0.584